# **TMBD API Extraction**

_John Andrew Dixon_

---

##### **Imports**

In [37]:
import json, os, time
import pandas as pd
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

##### **Helper Functions**

In [38]:
def get_movie_with_rating(movie_id):
    """Using the TMDB API, retrieves movie info and rating based on a given movie_id"""

    # Using the TMDB API, get movie based on "movie_id"
    movie = tmdb.Movies(movie_id)

    # Save movie info to a variable
    info = movie.info()
    # Save movie releases to a variable
    releases = movie.releases()

    # Loop through all releases for each country
    for country in releases["countries"]:
        # If the country is US, add the certification to the info dictionary
        if country["iso_3166_1"] == "US":
            info["certification"] = country["certification"]

    # Return the info dictionary
    return info

In [39]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename, "r+") as file:
        # Load existing data as a dictionary
        file_data = json.load(file)

        # Choose to extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)

        # Set file's current position at offset
        file.seek(0)

        # Convert back to JSON
        json.dump(file_data, file)

##### **Data Load**

In [40]:
# Load the previous "basics" file from part 1
basics_df = pd.read_csv("Data/title_basics.csv.gz")
# Verify it loaded
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


---

## **Test API Function**

In [41]:
# Load API Key
with open("/Users/johna/.secret/tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [42]:
# Link API Key with API
tmdb.API_KEY = login["api-key"]

In [43]:
# IDs to test. The first ID = "Avengers"; the last ID = "The Notebook"
# The middle id = error test for a non-existent movie
test_ids = ["tt0848228", "tt0115937","tt0332280"]
# List to hold the results
results = []
# List to hold errors given by the API
errors = []

# Iterate through the test IDs
for movie_id in test_ids:
    # Try to get the current test_id's movie with the get_movie_with_rating function
    try:
        # Save the result of the function into movie_info
        movie_info = get_movie_with_rating(movie_id)
        # Append result to the results list
        results.append(movie_info)
        
    # Handle errors if movie is not found
    except Exception as e:
        # Append error to errors list
        errors.append((movie_id, e))
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.708,28570,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.879,10319,PG-13


In [44]:
# Display the errors
errors

[('tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=48b4a37e995f85ebabd7bd2be6b4d687'))]

This works, so let's move forward.

## **Efficient TMBD API Extraction**

In [45]:
# Specify the folder where data will be saved
FOLDER = "Data/"
# Create the folder; if it exists, move on
os.makedirs(FOLDER, exist_ok=True)
# List folder's contents
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [46]:
# Specify years to get
YEARS_TO_GET = list(range(2002, 2011))
# Create empty list to hold potential errors
errors = []

In [47]:
############################## OUTER LOOP ############################## 
# Loop through the years and display progress as progress bar
for YEAR in tqdm_notebook(YEARS_TO_GET, desc="Years", position=0):
    # Create the path to the JSON file for the current YEAR in FOLDER
    JSON_FILE = f"{FOLDER}tmdb_api_results_{YEAR}.json"

    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    # If the file does not exists, create in the OS and populate it
    if not file_exists:
        with open(JSON_FILE, "w") as file:
            json.dump([{"imdb_id": 0}], file)

    # Filter the "basics_df" by the current YEAR
    current_year_df = basics_df.loc[basics_df["startYear"] == YEAR].copy()
    # Get the movie IDs from movies of the current year
    movie_ids = current_year_df["tconst"].copy()

    # Load in previous results
    previous_results_df = pd.read_json(JSON_FILE)
    # Filter out the movie ids to get based on what was previous gotten
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_results_df["imdb_id"])]

    ############################## INNER LOOP ############################## 
    # Inner loop to iterate through the movie we need per year of the outer loop
    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f"Movies from {YEAR}", position=1, leave=True):
        try:
            # Get movie data for the current movie_id
            current_movie = get_movie_with_rating(movie_id)

            # Write the movie data to the proper file based on its year
            write_json(current_movie, JSON_FILE)

            # Wait 20 ms to prevent API overload
            time.sleep(0.02)
        # If there is an error, handle it
        except Exception as error:
            # Append all errors to the errors list
            errors.append([movie_id, error])
    
    # Read in the current year's API results from the file written to in the inner loop
    final_year_df = pd.read_json(JSON_FILE)
    # Compress the JSON file just created into a smaller format
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

Years:   0%|          | 0/9 [00:00<?, ?it/s]

Movies from 2002:   0%|          | 0/1558 [00:00<?, ?it/s]

Movies from 2003:   0%|          | 0/1678 [00:00<?, ?it/s]

Movies from 2004:   0%|          | 0/1895 [00:00<?, ?it/s]

Movies from 2005:   0%|          | 0/2178 [00:00<?, ?it/s]

Movies from 2006:   0%|          | 0/2424 [00:00<?, ?it/s]

Movies from 2007:   0%|          | 0/2566 [00:00<?, ?it/s]

Movies from 2008:   0%|          | 0/2903 [00:00<?, ?it/s]

Movies from 2009:   0%|          | 0/3545 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3848 [00:00<?, ?it/s]

In [48]:
# Output the errors from the results
print(f"[i] Total errors: {len(errors)}")

[i] Total errors: 5769
